In [12]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pickle
import numpy as np

import py21cmfast as p21c
from astropy.cosmology import Planck18

sys.path.append("..")
from dm21cm.dm_params import DMParams
from dm21cm.evolve import evolve
import dm21cm.physics as phys

sys.path.append(os.environ['DH_DIR'])
from darkhistory.main import evolve as evolve_DH

## DMParam

In [13]:
dm_params = DMParams(
    mode = 'swave',
    primary = 'phot_delta',
    m_DM = 1e1,
    sigmav = 1e-50,
    struct_boost_model = 'erfc 1e-3',
)

## DarkHistory

In [14]:
soln = evolve_DH(
    DM_process=dm_params.mode, mDM=dm_params.m_DM,
    primary=dm_params.primary,
    sigmav=dm_params.sigmav, lifetime=dm_params.lifetime,
    struct_boost=dm_params.struct_boost,
    start_rs=3000, end_rs=5.+1, coarsen_factor=12, verbose=1,
    reion_switch=False
)

****** Loading transfer functions... ******
Using data at /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1
    for propagating photons...  

 Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******
****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******
Loading time: 34.676 s


  0%|          | 0/518 [00:00<?, ?it/s]

Initialization time: 0.070 s


100%|██████████| 518/518 [01:33<00:00,  5.54it/s]


Main loop time: 93.483 s


In [15]:
save_dir = "../data/run_info/DH"
pickle.dump(soln, open(f"{save_dir}/base_soln.p", 'wb'))

In [16]:
return_dict = evolve(
    run_name = 'base',
    z_start = 45.,
    z_end = 5.,
    zplusone_step_factor = 1.01,
    dm_params = dm_params,
    enable_elec = True,
    tf_version = '230629',
    
    p21c_initial_conditions = p21c.initial_conditions(
        user_params = p21c.UserParams(
            HII_DIM = 64,
            BOX_LEN = 64 * 2, # [conformal Mpc]
            N_THREADS = 32,
        ),
        cosmo_params = p21c.CosmoParams(
            OMm = Planck18.Om0,
            OMb = Planck18.Ob0,
            POWER_INDEX = Planck18.meta['n'],
            SIGMA_8 = Planck18.meta['sigma8'],
            hlittle = Planck18.h,
        ),
        random_seed = 54321,
        write = True,
    ),
    
    rerun_DH = True,
    clear_cache = True,
    use_tqdm = True,
)

/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/inputs.py:487: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(
INFO:root:Using 21cmFAST version 0.1.dev1579+g6b1da6d.d20230914
INFO:root:Cache dir: /n/holyscratch01/iaifi_lab/yitians/21cmFAST-cache/base
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Interpreter CUDA
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO:root:TransferFunctionWrapper: Loaded photon transfer functions.
INFO:root:TransferFunctionWrapper: Loaded electron transfer functions.

Loading time: 0.000 s


  1%|          | 2/357 [00:00<00:34, 10.37it/s]

Initialization time: 0.023 s


100%|██████████| 357/357 [01:02<00:00,  5.69it/s]


Main loop time: 62.770 s


INFO:root:DarkHistoryWrapper: Saved DarkHistory initial conditions.
  1%|          | 2/205 [00:08<13:53,  4.11s/it]


ValueError: input_heating.input_heating has NaNs

## compare

In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../matplotlibrc')

In [3]:
dh_soln = pickle.load(open(f"../data/run_info/DH/ee_swave_soln.p", 'rb'))
soln = np.load(f"../data/run_info/ee_swave_records.npy", allow_pickle=True).item()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

ax = axs[0]
ax.plot(dh_soln['rs'] - 1, dh_soln['Tm']/phys.kB, label='DarkHistory')
ax.plot(soln['z'], soln['T_k'], label='dm21cm')
ax.set(xlim=(5, 50), xscale='log', yscale='log')
ax.set(xlabel='z', ylabel='T [K]')

ax = axs[1]
ax.plot(dh_soln['rs'] - 1, dh_soln['x'][:, 0], label='DarkHistory')
ax.plot(soln['z'], soln['x_e'], label='dm21cm')
ax.set(xlim=(5, 50))

## DEBUG